In [ ]:
"""
Author: Lay Monica Ratna Dewi
Correspondence email: lay.dewi@seh.ox.ac.uk
It takes about 15 minutes to run the whole code.
-----------------------------------------------------------------------------------------------------------------------------------------
This is a Python program for processing raw data from The Global Soil Mycobiome consortium (GSMc) dataset.
The GSMc dataset contains data for fungal species distribution in 108 countries, extracted from 3200 plots published by multiple studies.
The dataset could be downloaded from the links below (see https://link.springer.com/article/10.1007/s13225-021-00493-7#additional-information for the reference study):

1. The GSMc OTU-by-sample dataset is available from the PlutoF data repository (https://doi.org/10.15156/BIO/2263453) in six files:
    a. OTU matrix in spreadsheet format (Tedersoo L, Mikryukov V, Anslan S et al. Fungi_GSMc_taxonomy-function table_final.xlsx);
    b. Sample metadata (equivalent to Table S1) (Tedersoo L, Mikryukov V, Anslan S et al. Fungi_GSMc_sample_metadata.txt);
    c. Taxonomic and functional description of OTUs (Fungi_GSMc_OTU_Table.txt);
    d. OTU sequences in FASTA format (Tedersoo L, Mikryukov V, Anslan S et al. Fungi_GSMc_OTUs_fasta.fasta);
    e. Data in Biological Observation Matrix (BIOM) format (Tedersoo L, Mikryukov V, Anslan S et al. Fungi_GSMc_data_biom.biom).
    f. Information file (readme.txt).
2. Table S1 of the study (for easier data processing, use this one instead of file 1b):
    a. Download Supplementary File 1 (13225_2021_493_MOESM1_ESM.xlsx) from https://link.springer.com/article/10.1007/s13225-021-00493-7#Sec16.
-----------------------------------------------------------------------------------------------------------------------------------------
This code was created for and can be used for free for academic and educational purposes. No commercial use of this code is permitted. Recommended citation for this code:

Dewi, LMR. Visualising Distribution of Fungal Species from The Global Soil Mycobiome consortium (GSMc) Dataset. Version 1, 2025. [Python Source Code]. Accessed: [Month. Day, Year]. Available: [GitHub Link].
"""

In [ ]:
""" Step 1: How I setup Python environment and project
Note: Since the Python environment has come as one package with the GitHub repository, users do not need to perform these steps. The documentation below is only for illustration and recommendations for setting up new project.
 1. Open PyCharm
 2. In the Welcome page, choose "New Project" and set up the project directory
 3. Go to PyCharm Terminal (look for |>_| symbol)
 4. Run the following commands in PyCharm terminal: (How to run them all at once?)
    pip install biom-format (to install Biom library in order to process biom files)
    pip install pandas (to install pandas library in order to process tables and dataframes)
    pip install numpy (to install numpy library in order to perform mathematical operations)
"""

In [ ]:
# to measure the time required to run the code
import time
start = time.perf_counter()
start

In [ ]:
""" Step 2: Import the required libraries """
# to process BIOM table
import biom
from biom import example_table

# to process tables and dataframes
import pandas as pd

# to map the data points agains the map coordinates
import geopandas as gpd

# to produce the map
import folium as fol

# check if the biom package has been successfully installed by loading the example table
if print(example_table) != "":
    print("Biom was successfully imported!")
else:
    print("Biom was unsuccessfully imported!")

In [ ]:
""" Step 3: Familiarising oneself with the files """
# based on the dataset information in readme.txt, we only need to work with 3 files to achieve the objective
# the biom file contains information on species distribution by species code and country code, Table S1 contains the metadata for the country code, and the taxonomy-function excel table contains the mapping of species name for each of the species code
# since this is a Python project, user does not need to define the directory in this chunk as long as they store the files and notebook files in the same directory
# the files were downloaded from the GSMc data repository and have been stored in the same folder by the code author
# define functions that read the files

# define a function that reads the biom file
def load_biom_table(input_biom_file):
    if input_biom_file.endswith(".biom"):
        biom_dataset_table = biom.load_table(f"{input_biom_file}")
        biom_dataset_df = biom_dataset_table.to_dataframe()
        return biom_dataset_df
    elif not input_biom_file.endswith(".biom") == "":
        return "Please input the correct file format!"
    else:
        return "I expect at least one file to be specified!"

# define a function that reads the excel files
def load_excel_table(input_excel_file):
    if input_excel_file.endswith(".xlsx"):
        excel_dataset_table = pd.read_excel(f"{input_excel_file}")
        return excel_dataset_table
    elif not input_excel_file.endswith(".xlsx") == "":
        return "Please input the correct file format!"
    else:
        return "I expect at least one file to be specified!"

In [ ]:
# define and assign the files to variables (variables for the file-reading functions inputs), which can be modified as necessary by the user
fungal_biom_file = "Tedersoo L, Mikryukov V, Anslan S et al. Fungi_GSMc_data_biom.biom" #default input
metadata_excel_file = "13225_2021_493_MOESM1_ESM.xlsx"
taxonomy_excel_file = "Tedersoo L, Mikryukov V, Anslan S et al. Fungi_GSMc_taxonomy-function table_final.xlsx"

In [ ]:
# load the biom table with load_biom_table function
fungal_species_distribution = load_biom_table(fungal_biom_file)
fungal_species_distribution

In [ ]:
# load the metadata excel file with load_excel_table function
metadata = load_excel_table(metadata_excel_file)

# create a new column containing country name only rather than country:subnational regions
metadata["country"] = metadata["locality"].str.rsplit(":").str[0]
metadata

In [ ]:
# load the taxonomy excel file with load_excel_table function
taxonomy_table = load_excel_table(taxonomy_excel_file)
taxonomy_table

In [ ]:
# check unique name of countries from the column created in a previous ste
metadata["country"].unique()

In [ ]:
# define a function to filter the metadata based on the country of interest
def filter_by_country(dataframe, column, countries):
    metadata_filtered = dataframe[dataframe[column].isin(countries)]
    return metadata_filtered

# define a function to check if there is any missing values for the data of interest
def check_missing_data(dataframe, column):
    missing_data = dataframe[dataframe[column].isnull()].count().sum()
    return missing_data

# define a function to check unique values of the data of interest
def check_unique_values(dataframe, column):
    data_content = dataframe[column].unique()
    return data_content

In [ ]:
# load the list of asian countries from the excel file
countries = pd.read_excel("asian countries v3.xlsx")
countries = countries[countries["Southeast_Asia?"] == "Yes"]
countries = list(countries.iloc[:,1])
countries

In [ ]:
# plot name could correspond to country and location
column_name = "country"
countries_name = countries

# filter out data that do not belong to Southeast Asian countries
country_level_table = filter_by_country(metadata, column_name, countries_name)
country_level_table.head()


In [ ]:
# produce a list of codes corresponding to areas belonging to the Southeast Asian countries
country_plot_code = list(country_level_table['plot_name'])
country_plot_code

In [ ]:
# filter out the plot codes that do not belong to the Southeast Asian countries
country_fungal_species = fungal_species_distribution.filter(items = country_plot_code)

# sum the values of the columns to identify which samples are identified in the respective countries
country_fungal_species['samples_frequency'] = country_fungal_species.iloc[:,0:3].sum(axis = 1)

# filter out samples that are not identified in the countries
country_fungal_species_identified_only = country_fungal_species[country_fungal_species['samples_frequency']>0]
country_fungal_species_identified_only

In [ ]:
# insert row index so the Operational Taxonomic Units (OTU) now becomes a column
country_fungal_species_identified_only = country_fungal_species_identified_only.reset_index()
country_fungal_species_identified_only = country_fungal_species_identified_only.rename(columns={'index':'OTU'})
country_fungal_species_identified_only

In [ ]:
# list the identified OTU
list_of_identified_OTU = list(country_fungal_species_identified_only["OTU"])
list_of_identified_OTU

In [ ]:
#check for missing data
column_name = "phylum"
check_missing_data(taxonomy_table, column_name)

In [ ]:
#check for unique values for phylum
column_name = "phylum"
check_unique_values(taxonomy_table, column_name)

In [ ]:
# Filter out OTUs that are not identified in the countries of interest
taxonomy_table_filtered = taxonomy_table[taxonomy_table["OTU"].isin(list_of_identified_OTU)]
taxonomy_table_filtered

In [ ]:
# join the tables to connect the OTUs in fungal_species_distribution with the Phylum in taxonomy_table
# to avoid dealing with larger amount of data than we should be, first we are going to filter OTUs and Phylum columns from the taxonomy_table
selected_columns = ["OTU","phylum", "genus"]
taxonomy_table_identified_OTUs_phylum = taxonomy_table_filtered.filter(items = selected_columns)
taxonomy_table_identified_OTUs_phylum

In [ ]:
# filter out columns that are not needed
plot_coordinates = country_level_table.filter(items = ["plot_name", "country", "latitude", "longitude"],
                                              axis = 1)
plot_coordinates

In [ ]:
# merge the taxonomy table or country fungal species table
merged_table_OTU_phylum = pd.merge(taxonomy_table_identified_OTUs_phylum,
                                   country_fungal_species_identified_only,
                                   left_on="OTU",
                                   right_on="OTU")

merged_table_OTU_phylum

In [ ]:
# transform the merged table into the wide format
wide_format_merged_table_OTU_phylum = pd.melt(merged_table_OTU_phylum,
        id_vars = ["OTU","phylum", "genus", "samples_frequency"],
        value_vars = country_plot_code,
        var_name= "plot_name")

wide_format_merged_table_OTU_phylum

In [ ]:
# pivot the table to be a format the geopandas will be happy with
final_table_for_pivot = pd.merge(plot_coordinates,
                                 wide_format_merged_table_OTU_phylum,
                                 left_on = "plot_name",
                                 right_on = "plot_name")
final_table_for_pivot

In [ ]:
# pivot the table to be a format geopandas will be happy with
country_pivot_by_location = pd.pivot_table(final_table_for_pivot,
            values = ["value"],
            index = ["phylum", "genus", "latitude", "longitude"],
            aggfunc = "sum").reset_index()
country_pivot_by_location = country_pivot_by_location[country_pivot_by_location["latitude"] != "unknown"]
country_pivot_by_location

In [ ]:
# assign geopoints to the table based on the data point coordinates
geo_df_for_plotting = gpd.GeoDataFrame(
    country_pivot_by_location,
    geometry = gpd.points_from_xy(country_pivot_by_location["longitude"],
                                        country_pivot_by_location["latitude"]),
    crs = "EPSG:4326"
)
geo_df_for_plotting

In [ ]:
# code modified from https://github.com/catris25/Indonesian-Volcanoes-in-Folium and https://geopandas.org/en/stable/

# define the region map based on the mean values of the latitudes and longitudes
region_map = [geo_df_for_plotting['latitude'].mean(), geo_df_for_plotting['longitude'].mean()]

# define the map attributes with folium
map1 = fol.Map(location = region_map,
                  tiles='Openstreetmap',
                  zoom_start = 4,
                  control_scale=True)

# create a function that takes data frame and taxa level as inputs and plot the data points to the region map
def add_points_to_map(geo_dataframe, taxa_level):
    for index, loc in geo_dataframe.iterrows():
        fol.CircleMarker([loc['latitude'],
                             loc['longitude']],
                            radius=2, weight=4,
                            popup=loc[taxa_level]).add_to(map1)
    fol.LayerControl().add_to(map1)
    return map1

# setup the function input and plot the map
taxa = "phylum"
add_points_to_map(geo_df_for_plotting, taxa)

In [ ]:
# code extension: filter data points by genus
map2 = fol.Map(location = region_map,
                  tiles='Openstreetmap',
                  zoom_start = 5,
                  control_scale=True)

# define a function that takes data frame, taxa level, and the name of the respective group we are interested in
# the output should be a map with the data points we are interested in and we filter
def plot_different_group(geo_dataframe, taxa_level, group_name):
    for index, loc in geo_dataframe.iterrows():
        if loc[taxa_level]== group_name[0]:
            color = 'red'
        elif loc[taxa_level]== group_name[1]:
            color = "green"
        elif loc[taxa_level]== group_name[2]:
            color = "blue"
        else:
            color = 'black'
        fol.RegularPolygonMarker([loc['latitude'],
                                     loc['longitude']],
                                    popup = loc[taxa_level],
                                    fill_color=color,
                                    number_of_sides=3,
                                    radius=6,
                                    rotation=30).add_to(map2)
    fol.LayerControl().add_to(map2)
    return map2

# set the functions input and plot the map
taxa = "genus"
group = ['Acremonium', 'Albonectria', 'Arnium']
plot_different_group(geo_df_for_plotting, taxa, group)

In [ ]:
# see how long it takes to execute the whole code
end = time.perf_counter()
(end-start)/60